In [1]:
import os                       # for working with files
import numpy as np              # for numerical computationss
import pandas as pd             # for working with dataframes
import seaborn as sns
import torch                    # Pytorch module 
import matplotlib.pyplot as plt # for plotting informations on graph and images using tensors
from torch.utils.data import DataLoader # for dataloaders 
from PIL import Image           # for checking images
import tensorflow as ts 
from  tensorflow import keras
import itertools
from sklearn.metrics import precision_score, accuracy_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

%matplotlib inline

In [2]:
data_dir = "/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)"
train_dir = data_dir + "/train"
valid_dir = data_dir + "/valid"
diseases = os.listdir(train_dir)

In [3]:
diseases

['Tomato___Late_blight',
 'Tomato___healthy',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Potato___healthy',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Tomato___Early_blight',
 'Tomato___Septoria_leaf_spot',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Strawberry___Leaf_scorch',
 'Peach___healthy',
 'Apple___Apple_scab',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Bacterial_spot',
 'Apple___Black_rot',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Peach___Bacterial_spot',
 'Apple___Cedar_apple_rust',
 'Tomato___Target_Spot',
 'Pepper,_bell___healthy',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Potato___Late_blight',
 'Tomato___Tomato_mosaic_virus',
 'Strawberry___healthy',
 'Apple___healthy',
 'Grape___Black_rot',
 'Potato___Early_blight',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Common_rust_',
 'Grape___Esca_(Black_Measles)',
 'Raspberry___healthy'

In [4]:
train_data =keras.utils.image_dataset_from_directory(train_dir ,
                                         image_size=(256, 256))
valid_data = keras.utils.image_dataset_from_directory(valid_dir,
                                        image_size=(256, 256))

Found 70295 files belonging to 38 classes.
Found 17572 files belonging to 38 classes.


In [5]:
image_size = 128

In [6]:
rescale = keras.layers.Rescaling(scale=1.0/255)
resize = keras.layers.Resizing(image_size,image_size)

In [7]:
train_gen = train_data.map(lambda image,label:(resize(rescale(image)),label))
valid_gen  = valid_data.map(lambda image,label:(resize(rescale(image)),label))

In [8]:
valid_genx = np.array([img_r for img_r, label in valid_gen]).flatten()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [9]:
valid_data.map(lambda image,label:(resize(rescale(image)),label))

<MapDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [10]:
import keras
import tensorflow as tf
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Resizing, Input, Dense, Dropout, GRU, Flatten, Input, Conv2D, MaxPooling2D, Reshape, Concatenate
from keras.optimizers import Adam, SGD
from keras.layers import Reshape
from keras.models import Model

In [11]:
input_shape = (image_size, image_size, 3)
input_1 = Input(shape=input_shape, name='input_1')

# gru = MaxPooling2D((2 , 2))(input_1)
gru = Reshape((image_size,-1))(input_1)
gru = GRU(64, input_shape=(image_size, 96))(gru)
gru = Flatten()(gru)

# cnn = MaxPooling2D((2 , 2))(input_1)
cnn = Conv2D(image_size,(3 ,3), activation='relu')(input_1)
cnn = MaxPooling2D((2 , 2))(cnn)
cnn = Flatten()(cnn)

concatenated = Concatenate()([cnn, gru]) 

fc = Dense(256, activation='relu')(concatenated)
fc = Dropout(0.5)(fc)
fc = Dense(38, activation='softmax')(fc)
model= Model(inputs=input_1, outputs=fc)

In [12]:
# Compile the model
optimizer = SGD(lr=0.01, momentum=0.85, nesterov=True)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 126, 126, 12  3584        ['input_1[0][0]']                
                                8)                                                                
                                                                                                  
 reshape (Reshape)              (None, 128, 384)     0           ['input_1[0][0]']                
                                                                                              

/opt/conda/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [13]:
# Train the model
history = model.fit(train_gen,
          validation_data=valid_gen,
          epochs = 10,
          batch_size=32)

Epoch 1/10
2197/2197 [==============================] - 286s 127ms/step - loss: 1.9371 - accuracy: 0.4378 - val_loss: 1.0276 - val_accuracy: 0.6901
Epoch 2/10
2197/2197 [==============================] - 124s 56ms/step - loss: 1.1898 - accuracy: 0.6363 - val_loss: 0.7538 - val_accuracy: 0.7587
Epoch 3/10
2197/2197 [==============================] - 131s 60ms/step - loss: 0.9470 - accuracy: 0.7080 - val_loss: 0.6748 - val_accuracy: 0.7883
Epoch 4/10
2197/2197 [==============================] - 149s 68ms/step - loss: 0.8155 - accuracy: 0.7480 - val_loss: 0.5892 - val_accuracy: 0.8139
Epoch 5/10
2197/2197 [==============================] - 144s 65ms/step - loss: 0.7109 - accuracy: 0.7788 - val_loss: 0.4906 - val_accuracy: 0.8452
Epoch 6/10
2197/2197 [==============================] - 122s 56ms/step - loss: 0.6350 - accuracy: 0.8013 - val_loss: 0.5497 - val_accuracy: 0.8318
Epoch 7/10
2197/2197 [==============================] - 122s 55ms/step - loss: 0.5560 - accuracy: 0.8261 - val_loss: 

In [14]:
from tensorflow.keras.models import load_model
m_saved = model.save("/kaggle/working/abc.h5")

## Tấn công

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.nn import *
#import tensorflow_model_optimization as tfmot
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import tensorboard
from datetime import datetime
import os
import cv2
import tempfile

from PIL import Image
import numpy as np
from numpy import array
#from imutils import paths
import pathlib
from imageio import imwrite
import matplotlib.pyplot as plt


#import setGPU

In [16]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import cv2

mpl.rcParams['figure.figsize'] = (8, 8)
mpl.rcParams['axes.grid'] = False

In [17]:
model.trainable = False

In [18]:
for img, lab in valid_data.take(1):
    image = img[0]
    label = lab[0]
    break

In [ ]:
plt.figure()
plt.imshow(image / 255)  
plt.title(diseases[label])
plt.show()

In [ ]:
def create_adversarial_pattern(pretrained_model, input_image, input_label):
    with tf.GradientTape() as tape:
        tape.watch(input_image)
        
        if isinstance(input_label, np.uint8):
            target_vector = np.zeros((1,2))
            target_vector[0,input_label] = 1
        else:
            target_vector = np.copy(input_label)

        #forward propagate image and retrieve the loss
        prediction = pretrained_model(input_image)
        loss = tf.reduce_sum(tf.math.square(prediction-target_vector))

       # obtain the gradient of the loss with respect to the input image
        gradient = tape.gradient(loss, input_image)

       # Get the sign of the gradients (directions)
        signed_grad = tf.sign(gradient)
        return signed_grad


In [ ]:
def fgsm(target_model, x_train, labels, epsilon = 0.1):
    gen_img = tf.identity(x_train)
    gen_img = gen_img + tf.random.uniform(gen_img.get_shape().as_list(), minval=-epsilon, maxval=epsilon, dtype=tf.dtypes.float32)
    
    #alpha = epsilon/iterations
    if isinstance(labels, np.uint8):
      target_vector = np.zeros((1,2))
      target_vector[0,labels] = 1 
    else:
      target_vector = np.copy(labels)
    
    x_temp = x_train.numpy()
    
    imgv = tf.Variable(gen_img)
    with tf.GradientTape() as tape:
        tape.watch(imgv)
        predictions = model(imgv)
        loss = tf.reduce_sum(tf.math.square(predictions-target_vector))
            
    grads = tape.gradient(loss,imgv)
        
    signed_grads = tf.sign(grads)
        
    gen_img = gen_img + tf.math.sign(signed_grads)*epsilon
        
    gen_img = tf.clip_by_value(gen_img, x_train-epsilon, x_train+epsilon)
    
    return gen_img

In [ ]:
image.shape

In [ ]:
image = np.array([image])

In [ ]:
image= resize(image)

In [ ]:
image_probs = model.predict(image)

In [ ]:
image_probs

In [ ]:
# Get the input label of the image.
label = tf.cast(label, tf.int32 )

perturbations = create_adversarial_pattern(model, tf.convert_to_tensor(image / 255), label)
plt.imshow(perturbations[0] * 0.5 + 0.5);  # To change [-1, 1] to [0,1]

In [ ]:
def get_imagenet_label(probs):
  return diseases[np.argmax(probs)], np.max(probs)

In [ ]:
model.predict(image)

In [ ]:
def display_images(image, description):
  label, confidence = get_imagenet_label(model.predict((image+1)/2 )[0])
  plt.figure()
  plt.imshow(image[0]*0.5+0.5)
  plt.title('{} : {:.2f}% Confidence'.format(label, confidence*100))
  plt.show()

In [ ]:
image = image/255

In [ ]:
epsilons = [0, 0.01, 0.1, 0.15, 0.2, 1]
descriptions = [('Epsilon = {:0.3f}'.format(eps) if eps else 'Input')
                for eps in epsilons]

for i, eps in enumerate(epsilons):
  adv_x = image * 2 - 1 + eps*perturbations
  adv_x = tf.clip_by_value(adv_x, -1, 1)
  display_images(adv_x, descriptions[i])

# Tấn công toàn bộ ảnh

In [ ]:
def RESULT ( eps ):
    count = 0;
    for img, lab in valid_data:
        image = img[0]
        label = lab[0]


        image = np.array([image])
        image= resize(image)
        image_probs = model.predict(image)

        image1 = image/255

        epsilons = eps

        adv_x = image1 * 2 - 1 + epsilons*perturbations
        adv_x = tf.clip_by_value(adv_x, -1, 1)
        label1, confidence = get_imagenet_label1(model.predict((adv_x +1)/2 )[0])

        print(diseases[label])
        print(label1)

        if (label1 == diseases[label]):
            count = count +1
            
    return count/len(valid_data) 

In [ ]:
%%capture
arr = [0, 0.01, 0.1, 0.15]
res = []
for eps in arr:
    res.append(RESULT(eps))

In [ ]:
res

In [ ]:
def get_imagenet_label1(probs):
  return np.argmax(probs), np.max(probs)

In [ ]:
def Pertubation ( eps = 0.15):
    count = 0;
    perturbImages = []
    perturbLabels = []
    for img, lab in valid_data:
        image = img[0]
        label = lab[0]


        image = np.array([image])
        image= resize(image)
        image_probs = model.predict(image)

        image1 = image/255

        epsilons = eps

        adv_x = image1 * 2 - 1 + epsilons*perturbations
        adv_x = tf.clip_by_value(adv_x, -1, 1)
        label1, confidence = get_imagenet_label1(model.predict((adv_x +1)/2 )[0])
        
        # update our perturbed images and labels lists
        perturbImages.append(adv_x)
        perturbLabels.append(label1)
        
        
            
    return perturbImages,perturbLabels

In [ ]:
# Compile the model
optimizer = SGD(lr=0.01, momentum=0.85, nesterov=True)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

In [ ]:
#print(valid_gen)

for i in valid_gen:
    print(i)
    break

In [40]:
 perturbImages,perturbLabels = Pertubation()
# Train the model
history = model.fit(perturbImages,
                    perturbLabels,
          epochs = 10,
          batch_size=32)

1/1 [==============================] - 0s 22ms/step


ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'tensorflow.python.framework.ops.EagerTensor'>"}), (<class 'list'> containing values of types {"<class 'numpy.int64'>"})

In [ ]:
# now that our model is fine-tuned we should evaluate it on the test
# set (i.e., non-adversarial) again to see if performance has degraded
(loss, acc) = model.evaluate(x=[], verbose=0)
print("")
print("[INFO] normal testing images *after* fine-tuning:")
print("[INFO] loss: {:.4f}, acc: {:.4f}\n".format(loss, acc))
# do a final evaluation of the model on the adversarial images
(loss, acc) = model.evaluate(x=perturbImages, y=perturbLabels, verbose=0)
print("[INFO] adversarial images *after* fine-tuning:")
print("[INFO] loss: {:.4f}, acc: {:.4f}".format(loss, acc))